In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

In [2]:
year = 2024
loads = pd.read_csv(f'../data/Prepared data/capitl_{year}.csv')
weather = pd.read_csv(f'../data/Prepared data/{year}_weather_data.csv')

In [3]:
weather

,valid_time_gmt,temp,wspd,pressure,precip_hrly
0,2024-01-01 00:51,43.0,7.0,29.98,0.0
1,2024-01-01 01:51,43.0,9.0,29.97,0.0
2,2024-01-01 02:51,42.0,9.0,29.97,0.0
3,2024-01-01 03:51,42.0,7.0,29.96,0.0
4,2024-01-01 04:51,42.0,5.0,29.96,0.0
...,...,...,...,...,...
9706,2024-12-17 11:06,53.0,13.0,30.09,0.0
9707,2024-12-17 11:33,54.0,10.0,30.09,0.0
9708,2024-12-17 11:51,55.0,12.0,30.08,0.0
9709,2024-12-17 12:51,58.0,9.0,30.08,0.0


In [4]:
loads

,timestamp,name,ptid,load
0,01/01/2024 00:00:00,CAPITL,61757,1184.9514
1,01/01/2024 00:05:00,CAPITL,61757,1176.8291
2,01/01/2024 00:10:00,CAPITL,61757,1176.4692
3,01/01/2024 00:15:00,CAPITL,61757,1159.2637
4,01/01/2024 00:20:00,CAPITL,61757,1173.3499
...,...,...,...,...
103125,12/17/2024 14:10:00,CAPITL,61757,1255.1427
103126,12/17/2024 14:15:00,CAPITL,61757,1247.9932
103127,12/17/2024 14:20:00,CAPITL,61757,1254.9329
103128,12/17/2024 14:25:00,CAPITL,61757,1258.8540


In [5]:
weather = weather.rename(columns={"valid_time_gmt": "timestamp"})

In [6]:
# weather['date'] = weather.date.apply(lambda x: pd.to_datetime(x).date())
# weather['timeest'] = weather.Time.apply(lambda x: pd.to_datetime(x).time())
# foo = weather[['date', 'timeest']].astype(str)
# weather['timestamp'] = pd.to_datetime(foo['date'] + ' ' + foo['timeest'])
print(weather.columns)
weather['timestamp'] = weather.timestamp.apply(lambda x: pd.to_datetime(x))
loads['timestamp'] = loads.timestamp.apply(lambda x: pd.to_datetime(x))



Index(['timestamp', 'temp', 'wspd', 'pressure', 'precip_hrly'], dtype='object')


In [7]:
weather

,timestamp,temp,wspd,pressure,precip_hrly
0,2024-01-01 00:51:00,43.0,7.0,29.98,0.0
1,2024-01-01 01:51:00,43.0,9.0,29.97,0.0
2,2024-01-01 02:51:00,42.0,9.0,29.97,0.0
3,2024-01-01 03:51:00,42.0,7.0,29.96,0.0
4,2024-01-01 04:51:00,42.0,5.0,29.96,0.0
...,...,...,...,...,...
9706,2024-12-17 11:06:00,53.0,13.0,30.09,0.0
9707,2024-12-17 11:33:00,54.0,10.0,30.09,0.0
9708,2024-12-17 11:51:00,55.0,12.0,30.08,0.0
9709,2024-12-17 12:51:00,58.0,9.0,30.08,0.0


In [8]:
from sklearn.neighbors import NearestNeighbors

def find_nearest(group, match, groupname):
    nbrs = NearestNeighbors(n_neighbors=1).fit(match['timestamp'].values[:, None])
    dist, ind = nbrs.kneighbors(group['timestamp'].values[:, None])

    group['nearesttime'] = match['timestamp'].values[ind.ravel()]
    return group

In [9]:
loads = find_nearest(loads,weather,'timestamp')

In [10]:
loads

,timestamp,name,ptid,load,nearesttime
0,2024-01-01 00:00:00,CAPITL,61757,1184.9514,2024-01-01 00:51:00
1,2024-01-01 00:05:00,CAPITL,61757,1176.8291,2024-01-01 00:51:00
2,2024-01-01 00:10:00,CAPITL,61757,1176.4692,2024-01-01 00:51:00
3,2024-01-01 00:15:00,CAPITL,61757,1159.2637,2024-01-01 00:51:00
4,2024-01-01 00:20:00,CAPITL,61757,1173.3499,2024-01-01 00:51:00
...,...,...,...,...,...
103125,2024-12-17 14:10:00,CAPITL,61757,1255.1427,2024-12-17 13:51:00
103126,2024-12-17 14:15:00,CAPITL,61757,1247.9932,2024-12-17 13:51:00
103127,2024-12-17 14:20:00,CAPITL,61757,1254.9329,2024-12-17 13:51:00
103128,2024-12-17 14:25:00,CAPITL,61757,1258.8540,2024-12-17 13:51:00


In [11]:
full = loads.merge(weather, left_on='nearesttime', right_on='timestamp')
print(full.columns)
print(full)

full = full[['timestamp_x', 'load', 'nearesttime', 'temp', 'wspd', 'pressure', 'precip_hrly']].rename(columns={'timestamp_x': 'timestamp', 'nearesttime':'weathertime'})

Index(['timestamp_x', 'name', 'ptid', 'load', 'nearesttime', 'timestamp_y',
       'temp', 'wspd', 'pressure', 'precip_hrly'],
      dtype='object')
               timestamp_x    name   ptid       load         nearesttime  \
0      2024-01-01 00:00:00  CAPITL  61757  1184.9514 2024-01-01 00:51:00   
1      2024-01-01 00:05:00  CAPITL  61757  1176.8291 2024-01-01 00:51:00   
2      2024-01-01 00:10:00  CAPITL  61757  1176.4692 2024-01-01 00:51:00   
3      2024-01-01 00:15:00  CAPITL  61757  1159.2637 2024-01-01 00:51:00   
4      2024-01-01 00:20:00  CAPITL  61757  1173.3499 2024-01-01 00:51:00   
...                    ...     ...    ...        ...                 ...   
103125 2024-12-17 14:10:00  CAPITL  61757  1255.1427 2024-12-17 13:51:00   
103126 2024-12-17 14:15:00  CAPITL  61757  1247.9932 2024-12-17 13:51:00   
103127 2024-12-17 14:20:00  CAPITL  61757  1254.9329 2024-12-17 13:51:00   
103128 2024-12-17 14:25:00  CAPITL  61757  1258.8540 2024-12-17 13:51:00   
103129 2024-12-

In [12]:
pday = pd.Timedelta('1 day')

def get_prev_days(x, n_days):
    try:
        lo = full[full.timestamp == x - n_days*pday].load.values[0]
    except:
        lo = full[full.timestamp == x].load.values[0]
    return lo 

In [13]:
full['dow'] = full.timestamp.apply(lambda x: x.dayofweek)
full['doy'] = full.timestamp.apply(lambda x: x.dayofyear)
full['day'] = full.timestamp.apply(lambda x: x.day)
full['month'] = full.timestamp.apply(lambda x: x.month)
full['hour'] = full.timestamp.apply(lambda x: x.hour)
full['minute'] = full.timestamp.apply(lambda x: x.hour*60 + x.minute)
full['year'] = full.timestamp.apply(lambda x: x.year)

# full['t_m24'] = full.timestamp.apply(get_prev_days, args=(1,))
# full['t_m48'] = full.timestamp.apply(get_prev_days, args=(2,))
# full['tdif'] = full['load'] - full['t_m24']

In [14]:
# full['Temperature'] = full['Temperature'].map(lambda x: x[:-2])
# full['Dew Point'] = full['Dew Point'].map(lambda x: x[:-2])
# full['Humidity'] = full['Humidity'].map(lambda x: x[:-2])
# full['Pressure'] = full['Pressure'].map(lambda x: x[:-3])
# full['Wind Speed'] = full['Wind Speed'].map(lambda x: x[:-4])
# full['precipitation'] = full['precipitation'].map(lambda x: x[:-3])

In [15]:
full

,timestamp,load,weathertime,temp,wspd,pressure,precip_hrly,dow,doy,day,month,hour,minute,year
0,2024-01-01 00:00:00,1184.9514,2024-01-01 00:51:00,43.0,7.0,29.98,0.0,0,1,1,1,0,0,2024
1,2024-01-01 00:05:00,1176.8291,2024-01-01 00:51:00,43.0,7.0,29.98,0.0,0,1,1,1,0,5,2024
2,2024-01-01 00:10:00,1176.4692,2024-01-01 00:51:00,43.0,7.0,29.98,0.0,0,1,1,1,0,10,2024
3,2024-01-01 00:15:00,1159.2637,2024-01-01 00:51:00,43.0,7.0,29.98,0.0,0,1,1,1,0,15,2024
4,2024-01-01 00:20:00,1173.3499,2024-01-01 00:51:00,43.0,7.0,29.98,0.0,0,1,1,1,0,20,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103125,2024-12-17 14:10:00,1255.1427,2024-12-17 13:51:00,59.0,10.0,30.07,0.0,1,352,17,12,14,850,2024
103126,2024-12-17 14:15:00,1247.9932,2024-12-17 13:51:00,59.0,10.0,30.07,0.0,1,352,17,12,14,855,2024
103127,2024-12-17 14:20:00,1254.9329,2024-12-17 13:51:00,59.0,10.0,30.07,0.0,1,352,17,12,14,860,2024
103128,2024-12-17 14:25:00,1258.8540,2024-12-17 13:51:00,59.0,10.0,30.07,0.0,1,352,17,12,14,865,2024


In [16]:
full.to_csv(f'{year}_features.csv',encoding='utf-8-sig', index=False)